In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# FAISS, Facebook AI Similarity Search

1. 개요
    - Facebook AI에서 개발한 벡터 검색 및 유사도 매칭 라이브러리  
    - 임베딩된 데이터에서 ANN 검색을 수행하는데 최적화 되어 있음 
        - ANN, Approximate Nearest Neighbors  

2. 주요 특징 
    - 빠르고 확장 가능한 벡터 검색:
        - 대규모 데이터셋에서 고속으로 유사도 검색 수행  
        - CPU와 GPU를 활용한 효율적으로 동작 
    - 고차원 데이터 처리:
        - 100차원, 512차원, 1536차원 등 다양한 고차원 벡터를 처리하도록 설계  
        - ANN 검색에 계산 비용이 많이 요구되지만 속도와 정확도 간의 균형 맞춤 
    - 다양한 인덱싱:
        - 데이터 크기, 메모리 제한, 검색 정확도에 따라 인덱싱 전략 선택 가능 

3. 주요 인덱싱 유형
    - Flat Index:
        - 모든 벡터를 저장하고, 전체 데이터와 비교해 정확한 최근적 이웃 검색 
        - 가장 정확하지만 데이터 크기가 클수록 느리고 메모리 사용량 많음 
    - INF, Indexed Vectors File:
        - 벡터를 여러 클러스터로 나눠 검색 수행 
        - 클러스터별 제한된 범위에서 검색하여 속도가 빠르지만 약간의 정확도 손실 발생 
    - HNSW, Hierarchical Navigable Small World:
        - 그래프 기반 인덱싱 방법으로 ANN에서 높은 성능 보임 
        - 정확도와 검색 속도 간의 균형 좋음 
    - PQ, Product Quantization:
        - 벡터를 압축하여 메모리 사용량을 줄여 사용 
        - 대규모 데이터셋에 적합 


[Reference] https://faiss.ai

[Reference] https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/

# Vanilla

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

In [4]:
# 문서 데이터 (Document 객체로 정의)
documents = [
    Document(page_content="AI is transforming the world.", metadata={"id": 1}),
    Document(page_content="Machine learning is a subset of AI.", metadata={"id": 2}),
    Document(page_content="Deep learning enables powerful AI models.", metadata={"id": 3}),
]

In [5]:
# OpenAI 임베딩 설정
embeddings = OpenAIEmbeddings()

# 벡터스토어 생성
vector_store = FAISS.from_documents(documents, embeddings)

In [6]:
# 유다소 검색
query = "What is AI?"
results = vector_store.similarity_search(query, k=2)

# 검색 결과 출력
for result in results:
    print(f"Content: {result.page_content}, Metadata: {result.metadata}")

Content: AI is transforming the world., Metadata: {'id': 1}
Content: Machine learning is a subset of AI., Metadata: {'id': 2}


# 파일 기반

In [7]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.docstore.in_memory import InMemoryDocstore

In [8]:
# 파일 경로 저장 
file_path1 = './data/nlp-keywords.txt'
file_path2 = './data/finance-keywords.txt'

In [9]:
# 분할기 정의 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 파일 로딩 
loader1 = TextLoader(file_path1)
loader2 = TextLoader(file_path2)

# 텍스트 분할 
text_split1 = loader1.load_and_split(text_splitter)
text_split2 = loader2.load_and_split(text_splitter)

In [10]:
# 분할된 파일 개수 확인 
len(text_split1), len(text_split2)

(7, 4)

## DB 저장

In [11]:
# 벡터DB 정의 
vector_store = FAISS.from_documents(
    documents=text_split1,
    embedding=OpenAIEmbeddings()
)

In [12]:
# 저장된 문서ID 확인
vector_store.index_to_docstore_id

{0: '85321f4b-b540-411d-b968-4ec741534fa4',
 1: 'b4fdb677-8604-4bb0-9d4a-9bc56deb8655',
 2: '0a669cfa-30b7-404c-acd5-033592f9f527',
 3: '007a6713-def5-4ce8-82d2-f8e661ffa7f8',
 4: 'f534efdb-f514-4255-b8ff-e804859f0cde',
 5: '88b239a5-b93f-4c5f-bc5c-fc8b776471b6',
 6: '22c469c7-5cb1-41bd-b2e6-e69350381fd2'}

In [13]:
# 저장된 문서ID별 상세 정보 확인 
vector_store.docstore._dict

{'85321f4b-b540-411d-b968-4ec741534fa4': Document(id='85321f4b-b540-411d-b968-4ec741534fa4', metadata={'source': './data/nlp-keywords.txt'}, page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nTokenizer\n\n정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.\n예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nVectorStore\n\n정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.\n예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할

## 유사도 검색

`similarity_search` 메서드는 주어진 쿼리와 가장 유사한 문서들을 검색하는 기능 제공

1. 매개변수
    - `query` (str): 유사한 문서를 찾기 위한 검색 쿼리 텍스트
    - `k` (int): 반환할 문서 수. 기본값은 4
    - `filter` (Optional[Union[Callable, Dict[str, Any]]]): 메타데이터 필터링 함수 또는 딕셔너리. 기본값은 None
    - `fetch_k` (int): 필터링 전에 가져올 문서 수. 기본값은 20
    - `**kwargs`: 추가 키워드 인자

2. 반환값
    - `List[Document]`: 쿼리와 가장 유사한 문서 리스트

3. 주요 특징
    - `filter` : 메타데이터 기반으로 필터링
    - `fetch_k` : 검색할 문서 수 

4. 고려사항
    - 자주 사용되는 쿼리에 대해서는 결과를 캐싱하여 반복적인 검색 속도를 향상시킬 수 있음
    - `fetch_k`를 너무 크게 설정하면 검색 속도가 느려질 수 있음

In [14]:
# 유사도 검색
vector_store.similarity_search("TF IDF 에 대하여 알려줘")

[Document(id='007a6713-def5-4ce8-82d2-f8e661ffa7f8', metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니

In [15]:
# k 값 지정
vector_store.similarity_search("TF IDF 에 대하여 알려줘", k=1)

[Document(id='007a6713-def5-4ce8-82d2-f8e661ffa7f8', metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니

In [16]:
# filter 사용
vector_store.similarity_search(
    "TF IDF 에 대하여 알려줘", 
    filter={"source": file_path1}, # nlp-keywords.txt
    k=2,
)

[Document(id='007a6713-def5-4ce8-82d2-f8e661ffa7f8', metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니

In [17]:
# filter 사용
vector_store.similarity_search(
    "TF IDF 에 대하여 알려줘", 
    filter={"source": file_path2}, # finance-keywords.txt
    k=2,
)

[]

## 문서 추가 

`add_documents` 메서드는 벡터 저장소에 문서를 추가하거나 업데이트하는 기능 제공

In [50]:
# vector_store.add_documents(text_split2)

In [18]:
# 추가된 문서임을 구문하기 위해 ID를 구분하고자 함 
ids = [f"text2_doc_{i}" for i in range(len(text_split2))]

# DB에 문서 추가 
vector_store.add_documents(text_split2, ids=ids)

['text2_doc_0', 'text2_doc_1', 'text2_doc_2', 'text2_doc_3']

In [19]:
# 결과 확인 
vector_store.similarity_search("Blue Chip Stocks", k=1)

[Document(id='text2_doc_0', metadata={'source': './data/finance-keywords.txt'}, page_content='S&P 500\n\n정의: S&P 500은 미국 주식 시장에 상장된 500개의 대형 기업의 주가를 종합한 지수입니다. 이는 미국 경제와 주식 시장의 전반적인 상황을 나타내는 주요 지표로 사용됩니다.\n예시: 애플, 마이크로소프트, 아마존과 같은 대형 기술 기업들이 S&P 500에 포함되어 있습니다.\n연관키워드: 주식 시장, 지수, 대형주\n\nMarket Capitalization\n\n정의: 시가총액은 회사의 발행 주식 수와 현재 주가를 곱한 값으로, 회사의 전체 가치를 나타냅니다.\n예시: 애플의 시가총액이 2조 달러를 넘어서면서 S&P 500 지수에서 가장 큰 비중을 차지하게 되었습니다.\n연관키워드: 기업 가치, 주식, 투자\n\nDividend\n\n정의: 배당금은 기업이 주주들에게 이익의 일부를 현금으로 지급하는 것을 말합니다.\n예시: 코카콜라는 50년 이상 연속으로 배당금을 인상해온 S&P 500 기업 중 하나입니다.\n연관키워드: 주주 가치, 수익률, 투자 전략\n\nBlue Chip Stocks\n\n정의: 블루칩 주식은 재무적으로 안정적이고 오랜 기간 동안 꾸준한 실적을 보여온 대형 기업의 주식을 의미합니다.\n예시: 존슨앤존슨, 프록터앤갬블과 같은 기업들은 S&P 500에 포함된 대표적인 블루칩 주식입니다.\n연관키워드: 안정적 투자, 대형주, 배당주\n\nSector Rotation\n\n정의: 섹터 로테이션은 투자자들이 경제 사이클에 따라 다른 산업 섹터로 투자를 이동시키는 전략을 말합니다.\n예시: 경기 회복기에 투자자들이 기술주에서 금융주로 투자를 이동시키는 것이 섹터 로테이션의 한 예입니다.\n연관키워드: 투자 전략, 자산 배분, 경제 사이클\n\nEarnings Per Share (EPS)\n\n정의: 주당순이익(EPS)은 기업의 순이익을 발행 주식 수로 나눈 

In [20]:
len(vector_store.index_to_docstore_id)

11

# 문서 삭제 

`delete` 메서드는 벡터 저장소에서 지정된 ID에 해당하는 문서를 삭제하는 기능 제공

1. 주요 특징
    - 문서 ID 기반 삭제로 정확한 문서 관리가 가능
    - FAISS 인덱스와 문서 저장소 양쪽에서 삭제 수행
    - 삭제 후 인덱스 재정렬을 통해 데이터 일관성을 유지

2. 주의사항
    - 삭제 작업은 되돌릴 수 없으므로 신중하게 수행
    - 동시성 제어가 구현되어 있지 않아 다중 스레드 환경에서 주의 필요

In [21]:
# 결과 확인 
vector_store.similarity_search("Defensive Stocks", k=1)

[Document(id='text2_doc_3', metadata={'source': './data/finance-keywords.txt'}, page_content='정의: 경기순환주는 경제 상황에 따라 실적이 크게 변동하는 기업의 주식을 말합니다.\n예시: 포드, 제너럴 모터스와 같은 자동차 기업들은 S&P 500에 포함된 대표적인 경기순환주입니다.\n연관키워드: 경제 사이클, 섹터 분석, 투자 타이밍\n\nDefensive Stocks\n\n정의: 방어주는 경기 변동에 상관없이 안정적인 실적을 보이는 기업의 주식을 의미합니다.\n예시: 프록터앤갬블, 존슨앤존슨과 같은 생활필수품 기업들은 S&P 500 내 대표적인 방어주로 꼽힙니다.\n연관키워드: 안정적 수익, 저변동성, 리스크 관리')]

In [22]:
# ID로 삭제 
delete_ids = ['text2_doc_2', 'text2_doc_3']
vector_store.delete(delete_ids)

True

In [23]:
# 문서ID 개수 확인 
len(vector_store.index_to_docstore_id)

9

In [24]:
# 결과 재확인 
vector_store.similarity_search("Defensive Stocks", k=1)

[Document(id='text2_doc_1', metadata={'source': './data/finance-keywords.txt'}, page_content='Price-to-Earnings Ratio (P/E Ratio)\n\n정의: 주가수익비율(P/E)은 주가를 주당순이익으로 나눈 값으로, 기업의 가치를 평가하는 데 사용되는 지표입니다.\n예시: 아마존의 P/E 비율이 높은 것은 투자자들이 회사의 미래 성장 가능성을 높게 평가하고 있다는 것을 의미합니다.\n연관키워드: 주식 가치평가, 투자 분석, 성장주\n\nQuarterly Earnings Report\n\n정의: 분기별 실적 보고서는 기업이 3개월마다 발표하는 재무 성과와 사업 현황에 대한 보고서입니다.\n예시: 애플의 분기별 실적 발표는 전체 기술 섹터와 S&P 500 지수에 큰 영향을 미칩니다.\n연관키워드: 기업 실적, 투자자 관계, 재무 분석\n\nIndex Fund\n\n정의: 인덱스 펀드는 S&P 500과 같은 특정 지수의 구성과 성과를 그대로 추종하는 투자 상품입니다.\n예시: 바운가드 S&P 500 ETF는 S&P 500 지수를 추종하는 대표적인 인덱스 펀드입니다.\n연관키워드: 패시브 투자, ETF, 포트폴리오 관리\n\nMarket Weight\n\n정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.\n예시: 기술 섹터는 S&P 500 지수에서 가장 큰 시장 가중치를 차지하고 있습니다.\n연관키워드: 포트폴리오 구성, 섹터 분석, 자산 배분\n\nGrowth Stock\n\n정의: 성장주는 평균 이상의 높은 성장률을 보이는 기업의 주식을 의미합니다.\n예시: 페이스북(메타)과 같은 기술 기업들은 S&P 500에 포함된 대표적인 성장주로 꼽힙니다.\n연관키워드: 고성장 기업, 기술주, 투자 전략\n\nValue Stock\n\n정의: 가치주는 현재 시장 가치가 내재 가치보다 낮다고 평가되는 기업의 주식을 말합니다.\n예시: 워렌 버핏이 투자한 

## 검색기 변환

`as_retriever` 메서드는 현재 벡터 저장소를 기반으로 `VectorStoreRetriever` 객체를 생성하는 기능 제공

1. 매개변수
    - `**kwargs`: 검색 함수에 전달할 키워드 인자
       - `search_type` (Optional[str]): 검색 유형 (`similarity`, `mmr`, `similarity_score_threshold`)
       - `search_kwargs` (Optional[Dict]): 검색 함수에 전달할 추가 키워드 인자

2. 주요 기능
    1. 다양한 검색 유형 지원:
        - `similarity`: 유사도 기반 검색 (기본값)
        - `mmr`: Maximal Marginal Relevance 검색
        - `similarity_score_threshold`: 임계값 기반 유사도 검색

    2. 검색 매개변수 커스터마이징:
        - `k`: 반환할 문서 수
        - `score_threshold`: 유사도 점수 임계값
        - `fetch_k`: MMR 알고리즘에 전달할 문서 수
        - `lambda_mult`: MMR 다양성 조절 파라미터
        - `filter`: 문서 메타데이터 기반 필터링

3. 주의사항
    - 부적절한 매개변수 설정은 검색 성능이나 결과의 품질에 영향을 줄 수 있음
    - 대규모 데이터셋에서 높은 `k` 값 설정은 검색 시간을 증가 시킬 수 있음

In [25]:
# 검색기 변환 
retriever = vector_store.as_retriever()
retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(id='007a6713-def5-4ce8-82d2-f8e661ffa7f8', metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니

In [26]:
# MMR 검색 
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={
        "k": 6, # 반환 문서수 (default : 4)
        "fetch_k": 10, # MMR알고리즘에 전달할 문서수 (default : 20)
        "lambda_mult": 0.25, # MMR결과 다양성 조절 (0~1, default: 0.5)
    }
)

retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(id='007a6713-def5-4ce8-82d2-f8e661ffa7f8', metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니

In [27]:
# 상위 2개 문서만 검색되도록 
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={
        "k": 2, 
        "fetch_k": 10, 
    }
)

retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(id='007a6713-def5-4ce8-82d2-f8e661ffa7f8', metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니

In [28]:
# 유사도 & 임계값 기반 검색되도록
retriever = vector_store.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={
        "score_threshold": 0.75, 
    }
)

retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(id='007a6713-def5-4ce8-82d2-f8e661ffa7f8', metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니

In [29]:
# 가장 유사한 단일 문서만 검색되도록
retriever = vector_store.as_retriever(
    search_kwargs={
        "k": 1, 
    }
)

retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(id='007a6713-def5-4ce8-82d2-f8e661ffa7f8', metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니

In [30]:
# 메타데이터 활용하여 필터가 필요할 때 
retriever = vector_store.as_retriever(
    search_kwargs={
        "filter": {'source':'./data/nlp-keywords.txt'}, 
        "k": 2,
    }
)

retriever.invoke('FAISS에 대해서 설명해 주세요')

[Document(id='007a6713-def5-4ce8-82d2-f8e661ffa7f8', metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source\n\n정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.\n예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.\n연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링\n\nParser\n\n정의: 파서는 주어진 데이터(문자열, 파일 등)를 분석하여 구조화된 형태로 변환하는 도구입니다. 이는 프로그래밍 언어의 구문 분석이나 파일 데이터 처리에 사용됩니다.\n예시: HTML 문서를 구문 분석하여 웹 페이지의 DOM 구조를 생성하는 것은 파싱의 한 예입니다.\n연관키워드: 구문 분석, 컴파일러, 데이터 처리\n\nTF-IDF (Term Frequency-Inverse Document Frequency)\n\n정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니

-----
** End of Documents **